In [1]:
import datetime
import os
import pandas as pd
import xarray as xr
import numpy as np
import geopandas as gpd
import glob 
import gc
import dask

##
## set the directories in which your copies of the data are in the dirs.py file
from dirs import wrfdir, basedir, coorddir, snoteldir
from myutils import wrfread, metaread
savedir = f'{basedir}/data/tmp/'


/global/homes/c/cowherd/.conda/envs/fos/lib/python3.11/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


we want to create some data files that contain pre-computed values from the downcsaled GCM outputs you can save these in a tmp folder while working on the project -- they are much smaller than the full  dataset but are directly repetitive

1. snowMax.nc 
2. allcorrs.npy




In [ ]:
%%time

## based on code from Stefan Rahimi
date_start_hist, date_end_hist = "1980-09-01", "2014-08-31"
date_start_ssp, date_end_ssp = "2014-09-01", "2100-08-31"

domain = "d02"
lat1, lon1, z1, file = metaread(coorddir,domain)
coords2 = xr.open_dataset(file)
lon_wrf = lon1[0,:,:]
lat_wrf = lat1[0,:,:]
z_wrf = z1[0,:,:]
lat_wrf = xr.DataArray(lat_wrf, dims=["lat2d", "lon2d"])
lon_wrf = xr.DataArray(lon_wrf, dims=["lat2d", "lon2d"])
z_wrf = xr.DataArray(z_wrf, dims=["lat2d", "lon2d"])

coords = {'lat': (['lat2d','lon2d'], lat_wrf), \
                 'lon': (['lat2d','lon2d'], lon_wrf) }

gcms = ['cesm2','mpi-esm1-2-lr','cnrm-esm2-1',
        'ec-earth3-veg','fgoals-g3','ukesm1-0-ll',
        'canesm5','access-cm2','ec-earth3']

variants = ['r11i1p1f1','r7i1p1f1','r1i1p1f2',
            'r1i1p1f1','r1i1p1f1','r2i1p1f2',
            'r1i1p2f1','r5i1p1f1','r1i1p1f1',
            ]

calendar = ['365_day','proleptic_gregorian','proleptic_gregorian',
            'proleptic_gregorian','365_day','360_day',
             '365_day','proleptic_gregorian','proleptic_gregorian',
            ]

ssps = ['ssp370','ssp370','ssp370','ssp370',
        'ssp370','ssp370','ssp370','ssp370',
        'ssp370','ssp370','ssp370','ssp370',
        ]

print (len(ssps),len(calendar),len(variants),len(gcms))


snow_BC = {}
snowMax = []

keys = []

for count, igcm in enumerate(gcms):
    
    condition = ()
    
    bc = True
    var = 'snow'
    key = '%s_%s_%s' %(igcm,variants[count],ssps[count])
    var = _read_merge(wrfdir,domain,var,
                                        igcm,variants[count], \
                                        date_start_hist,date_start_ssp, \
                                        date_end_hist,date_end_ssp,ssps[count],bc,
            calendar[count])
    
    snow_BC[key] = var
    
    var_x = var.groupby(var.time.dt.year).max()
    snowMax.append(var_x)

    
snowMax = xr.concat(snowMax,dim='gcm')
snowMax.to_netcdf(f'{savedir}snowMax.nc')
gc.collect()